In [3]:
#@title 🔧 Install Required Libraries
!pip install notion-client python-dotenv --quiet


In [2]:
#@title 📦 Import Packages
from notion_client import Client
import os
from pprint import pprint
import json
from datetime import datetime


In [7]:
#@title 🔑 Enter Your Notion Integration Token
NOTION_TOKEN = "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"  #@param {type:"string"}
notion = Client(auth=NOTION_TOKEN)



In [87]:
#@title 🧰 Utility: Extract Notion ID from URL or String
import re
from urllib.parse import urlparse

def get_id_from_url(url_or_id: str) -> str:
    """
    Extracts and cleans a Notion Page or Database ID from various URL formats or a raw ID string.

    Args:
        url_or_id: A Notion URL (e.g., from 'Copy link') or a raw 32/36-character ID.

    Returns:
        A 32-character ID string without hyphens, or an empty string if no ID is found.
    """
    if not isinstance(url_or_id, str):
        return ""

    # Remove any hyphens and strip whitespace first
    cleaned_id = url_or_id.replace("-", "").strip()

    # If the cleaned string is already a valid 32-char hex ID, return it.
    # This handles cases where a raw ID is passed in.
    if re.fullmatch(r'[0-9a-f]{32}', cleaned_id):
        return cleaned_id

    # If it's a URL, parse it to find the ID in the path
    try:
        parsed_url = urlparse(url_or_id)
        path = parsed_url.path

        # The ID is usually the last part of the path, after the last hyphen.
        # Example Page URL: /My-Page-Title-239e9c25ecb081818182e3f0bb84e930
        # Example DB URL: /239e9c25ecb081eeae1befdd211854d6

        # Split the path by '-' and get the last element.
        path_parts = path.split('-')
        potential_id = path_parts[-1]

        # Clean this potential ID and validate it.
        cleaned_potential_id = potential_id.replace("-", "").strip()
        if re.fullmatch(r'[0-9a-f]{32}', cleaned_potential_id):
            return cleaned_potential_id

    except Exception:
        # If parsing fails or anything else goes wrong, return empty string
        return ""

    return ""


print("✅ ID extraction utility `get_id_from_url` is now defined.")

✅ ID extraction utility `get_id_from_url` is now defined.


In [88]:
#@title 🔑 Set Global Page and Database IDs from URLs
# This cell takes the URLs you provide, extracts the IDs using the
# `get_id_from_url` utility, and stores them in global variables
# for other cells to use.

# --- User Inputs ---
# Paste your full "Copy link" URLs here.
NOTION_PAGE_URL = "https://www.notion.so/Journal-2025-07-23-239e9c25ecb081818182e3f0bb84e930?source=copy_link"  #@param {type:"string"}
NOTION_DB_URL = "https://www.notion.so/239e9c25ecb081eeae1befdd211854d6?v=239e9c25ecb081a98e27000c4be8d3d2&source=copy_link"  #@param {type:"string"}

# --- Initialize Global ID Variables ---
NOTION_PAGE_ID = ""
NOTION_DB_ID = ""

# --- Process Page URL ---
# Make sure the 'get_id_from_url' function from the previous cell has been run
if NOTION_PAGE_URL:
    NOTION_PAGE_ID = get_id_from_url(NOTION_PAGE_URL)
    print(f"🔗 Page URL detected.")
    print(f"🔑 Extracted Page ID: {NOTION_PAGE_ID}")
else:
    print("⚠️ No Page URL was provided.")

print("-" * 40)

# --- Process Database URL ---
if NOTION_DB_URL:
    NOTION_DB_ID = get_id_from_url(NOTION_DB_URL)
    print(f"🔗 Database URL detected.")
    print(f"🔑 Extracted Database ID: {NOTION_DB_ID}")
else:
    print("⚠️ No Database URL was provided.")

print("\n✅ Global variables `NOTION_PAGE_ID` and `NOTION_DB_ID` are now set and can be used in other cells.")

🔗 Page URL detected.
🔑 Extracted Page ID: 239e9c25ecb081818182e3f0bb84e930
----------------------------------------
🔗 Database URL detected.
🔑 Extracted Database ID: 

✅ Global variables `NOTION_PAGE_ID` and `NOTION_DB_ID` are now set and can be used in other cells.


In [5]:
# # Optional: Load from .env if you're using it Do not run if you do not have setup
# from dotenv import load_dotenv
# load_dotenv()
# notion = Client(auth=os.getenv("NOTION_TOKEN"))


In [89]:
#@title 🔍 Test Connection
user_info = notion.users.list()
pprint(user_info)


{'has_more': False,
 'next_cursor': None,
 'object': 'list',
 'request_id': '85ff173c-d5a6-4ea8-99ff-e618c0649b33',
 'results': [{'avatar_url': None,
              'id': '3326814f-5914-4d6e-bc68-e619eaaf657e',
              'name': 'Tyrique Daniel',
              'object': 'user',
              'person': {'email': 'codingoni@gmail.com'},
              'type': 'person'},
             {'avatar_url': None,
              'bot': {},
              'id': 'f3e8fc01-40b0-4c02-a33f-cd0b71411599',
              'name': 'CyberOni',
              'object': 'user',
              'type': 'bot'},
             {'avatar_url': 'https://s3-us-west-2.amazonaws.com/public.notion-static.com/83b8499a-e916-440e-bef2-d7f180b51d74/0d75566e-c7d2-4954-a089-5cfb63df1462.png',
              'bot': {'owner': {'type': 'workspace', 'workspace': True},
                      'workspace_limits': {'max_file_upload_size_in_bytes': 5368709120},
                      'workspace_name': 'CyberOni'},
              'id': '4e99c9e

In [90]:
#@title 🧰 Utility: Pretty Print JSON
def pretty(obj):
    print(json.dumps(obj, indent=2))


In [91]:
#@title 🧰 Utility: Normalize Notion ID
def clean_id(raw_id: str) -> str:
    """Remove dashes if ID is copy-pasted from Notion URL"""
    return raw_id.replace("-", "").strip()


In [92]:
#@title 🧰 Utility: Error Handler
def safe_run(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print("⛔ Error:", str(e))


In [95]:
#@title 📁 Create New Database (and Set Global ID)
# This cell creates a new database and, on success, automatically
# updates the global NOTION_DB_ID variable for other cells to use.

# Use the globally set page ID as the parent for the new database.
parent_page_id = NOTION_PAGE_ID

# --- User-configurable parameters ---
db_title = "My New Automated DB"  #@param {type:"string"}
is_inline = True #@param {type:"boolean"}


# --- Check if the parent page ID is set ---
if not parent_page_id:
    print("❌ Cannot create database. `NOTION_PAGE_ID` is not set.")
    print("   Please run the 'Set Global Page and Database IDs' cell first.")
else:
    # --- Define the database schema ---
    database_schema = {
        "Name": {"title": {}},
        "Status": {"select": {"options": [
            {"name": "To Do", "color": "red"},
            {"name": "In Progress", "color": "yellow"},
            {"name": "Done", "color": "green"}
        ]}},
        "Tags": {"multi_select": {"options": [
            {"name": "Work", "color": "blue"},
            {"name": "Personal", "color": "purple"}
        ]}},
        "Due": {"date": {}}
    }

    # --- Make the API call to create the database ---
    print(f"⚙️ Creating database '{db_title}' inside page {parent_page_id}...")
    response = safe_run(notion.databases.create,
        parent={"type": "page_id", "page_id": parent_page_id},
        title=[{"type": "text", "text": {"content": db_title}}],
        properties=database_schema,
        is_inline=is_inline
    )

    # --- If successful, update the global DB ID variable ---
    if response and response.get("id"):
        new_db_id = response.get("id")
        print(f"\n✅ Database created successfully!")

        # Update the global variable
        NOTION_DB_ID = get_id_from_url(new_db_id) # Use our utility to clean the ID

        print(f"🔑 Global variable 'NOTION_DB_ID' has been updated to: {NOTION_DB_ID}")
        print(f"🔗 URL: {response.get('url')}")

        # Optional: pretty print the full response
        # pretty(response)
    else:
        print("\n⚠️ Failed to create the database.")

⚙️ Creating database 'My New Automated DB' inside page 239e9c25ecb081818182e3f0bb84e930...

✅ Database created successfully!
🔑 Global variable 'NOTION_DB_ID' has been updated to: 239e9c25ecb0816fbb56d2cc40fe9170
🔗 URL: https://www.notion.so/239e9c25ecb0816fbb56d2cc40fe9170


In [15]:
#@title 📝 Create Page in Database
database_id = NOTION_DB_ID
database_id = clean_id(database_id)

page_title = "Example Task"  #@param {type:"string"}
status_value = "To Do"  #@param {type:"string"}
tag_values = ["Work"]  #@param
due_date = "2025-08-01"  #@param {type:"string"}

response = safe_run(notion.pages.create,
    parent={"database_id": database_id},
    properties={
        "Name": {"title": [{"text": {"content": page_title}}]},
        "Status": {"select": {"name": status_value}},
        "Tags": {"multi_select": [{"name": tag} for tag in tag_values]},
        "Due": {"date": {"start": due_date}}
    }
)

if response:
    print("✅ Page created:")
    pretty(response)


✅ Page created:
{
  "object": "page",
  "id": "239e9c25-ecb0-8119-a7d5-c5b7222b7f00",
  "created_time": "2025-07-23T18:52:00.000Z",
  "last_edited_time": "2025-07-23T18:52:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-08-01",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "select": {
        "id": "e229b395-57e8-4cff-a19e-03d655f7c675",
        "name": "To Do",
        "color": "red"
      }
    },
    "Tags": {
      "id": "ztBX",
      "type": "multi_select",
  

In [17]:
#@title 📄 List All Pages in a Database
database_id = NOTION_DB_ID
database_id = clean_id(database_id)

response = safe_run(notion.databases.query, database_id=database_id)

if response and response.get("results"):
    print(f"✅ {len(response['results'])} page(s) found:")
    for page in response["results"]:
        title = page["properties"]["Name"]["title"]
        title_text = title[0]["text"]["content"] if title else "No Title"
        print(f"- {title_text} (ID: {page['id']})")
else:
    print("⚠️ No pages found or error occurred.")


✅ 1 page(s) found:
- Example Task (ID: 239e9c25-ecb0-8119-a7d5-c5b7222b7f00)


In [19]:
#@title 🔎 Filter Pages by Status
database_id = NOTION_DB_ID
status_filter = "To Do"  #@param {type:"string"}

database_id = clean_id(database_id)

response = safe_run(notion.databases.query,
    database_id=database_id,
    filter={
        "property": "Status",
        "select": {
            "equals": status_filter
        }
    }
)

if response and response.get("results"):
    print(f"✅ {len(response['results'])} matching pages found:")
    for page in response["results"]:
        title = page["properties"]["Name"]["title"]
        title_text = title[0]["text"]["content"] if title else "No Title"
        print(f"- {title_text} (ID: {page['id']})")
else:
    print("⚠️ No pages match the filter.")


✅ 1 matching pages found:
- Example Task (ID: 239e9c25-ecb0-8119-a7d5-c5b7222b7f00)


In [25]:
database_id = NOTION_DB_ID
database_id = clean_id(database_id)

response = safe_run(notion.databases.retrieve, database_id=database_id)
if response:
    print("Properties in this database:")
    for prop_name in response["properties"]:
        print(f"- {prop_name}")


Properties in this database:
- Due
- Status
- Tags
- Name


In [27]:
#@title 🟢 Search for a Page and Update its Properties
database_id = NOTION_DB_ID
search_title = "Example Task"  #@param {type:"string"}

new_status = "Done"  #@param {type:"string"}
new_tags = ["Personal"]  #@param
new_due_date = "2025-08-10"  #@param {type:"string"}

database_id = clean_id(database_id)

# Step 1: Query database for page(s) matching title
response = safe_run(notion.databases.query,
    database_id=database_id,
    filter={
        "property": "Name",
        "title": {
            "contains": search_title
        }
    }
)

if response and response.get("results"):
    page = response["results"][0]  # Pick first match
    page_id = page["id"]
    page_title = page["properties"]["Name"]["title"][0]["text"]["content"]
    print(f"✅ Found page: {page_title} (ID: {page_id})")

    # Step 2: Prepare update data using correct property keys
    update_data = {}

    if new_status:
        update_data["Status"] = {"select": {"name": new_status}}

    if new_tags:
        update_data["Tags"] = {"multi_select": [{"name": tag} for tag in new_tags]}

    if new_due_date:
        update_data["Due"] = {"date": {"start": new_due_date}}

    if update_data:
        update_response = safe_run(notion.pages.update,
            page_id=page_id,
            properties=update_data
        )
        if update_response:
            print("✅ Page updated:")
            pretty(update_response)
    else:
        print("⚠️ No fields to update.")
else:
    print(f"❌ No page found with title containing '{search_title}'")


✅ Found page: Example Task (ID: 239e9c25-ecb0-8119-a7d5-c5b7222b7f00)
✅ Page updated:
{
  "object": "page",
  "id": "239e9c25-ecb0-8119-a7d5-c5b7222b7f00",
  "created_time": "2025-07-23T18:52:00.000Z",
  "last_edited_time": "2025-07-23T18:56:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-08-10",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "select": {
        "id": "9456af81-f09a-4f36-91b5-18267bec9048",
        "name": "Done",
        "color": "green"
      }
  

In [28]:
#@title 🧾 Search Page by Title and Update Page Title
database_id = NOTION_DB_ID
search_title = "Example Task"  #@param {type:"string"}
new_title = "Updated Task Name"  #@param {type:"string"}

database_id = clean_id(database_id)

# Step 1: Find page by title
response = safe_run(notion.databases.query,
    database_id=database_id,
    filter={
        "property": "Name",
        "title": {
            "contains": search_title
        }
    }
)

if response and response.get("results"):
    page = response["results"][0]
    page_id = page["id"]
    old_title = page["properties"]["Name"]["title"][0]["text"]["content"]
    print(f"✅ Found page: {old_title} (ID: {page_id})")

    # Step 2: Update the page title
    update_response = safe_run(notion.pages.update,
        page_id=page_id,
        properties={
            "Name": {
                "title": [{"text": {"content": new_title}}]
            }
        }
    )

    if update_response:
        print("✅ Page title updated.")
        pretty(update_response)
else:
    print(f"❌ No page found with title containing '{search_title}'")


✅ Found page: Example Task (ID: 239e9c25-ecb0-8119-a7d5-c5b7222b7f00)
✅ Page title updated.
{
  "object": "page",
  "id": "239e9c25-ecb0-8119-a7d5-c5b7222b7f00",
  "created_time": "2025-07-23T18:52:00.000Z",
  "last_edited_time": "2025-07-23T18:57:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-08-10",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "select": {
        "id": "9456af81-f09a-4f36-91b5-18267bec9048",
        "name": "Done",
        "color": "green"
    

In [29]:
#@title 🗃️ Search Page by Title and Archive (Soft Delete)
database_id = NOTION_DB_ID
search_title = "Updated Task Name"  #@param {type:"string"}

database_id = clean_id(database_id)

# Step 1: Query database for page(s) matching title
response = safe_run(notion.databases.query,
    database_id=database_id,
    filter={
        "property": "Name",
        "title": {
            "contains": search_title
        }
    }
)

if response and response.get("results"):
    page = response["results"][0]
    page_id = page["id"]
    page_title = page["properties"]["Name"]["title"][0]["text"]["content"]
    print(f"✅ Found page: {page_title} (ID: {page_id})")

    # Step 2: Archive the page
    archive_response = safe_run(notion.pages.update,
        page_id=page_id,
        archived=True
    )

    if archive_response:
        print("✅ Page archived (soft deleted).")
    else:
        print("⚠️ Could not archive the page.")
else:
    print(f"❌ No page found with title containing '{search_title}'")


✅ Found page: Updated Task Name (ID: 239e9c25-ecb0-8119-a7d5-c5b7222b7f00)
✅ Page archived (soft deleted).


In [32]:
# #@title 🔄 Search & Restore (Unarchive) Page by Title using `notion.search`
# search_title = "Updated Task Name"  #@param {type:"string"}

# response = safe_run(notion.search, query=search_title, filter={"property": "object", "value": "page"})

# if response and response.get("results"):
#     for page in response["results"]:
#         # Get the title from page properties (if exists)
#         props = page.get("properties", {})
#         name_prop = props.get("Name", {})
#         title_texts = name_prop.get("title", [])

#         if title_texts:
#             title = title_texts[0]["text"]["content"]
#             if search_title.lower() in title.lower():
#                 page_id = page["id"]
#                 print(f"✅ Found page: {title} (ID: {page_id})")

#                 # Restore (unarchive)
#                 restore_response = safe_run(notion.pages.update,
#                     page_id=page_id,
#                     archived=False
#                 )
#                 if restore_response:
#                     print("✅ Page restored.")
#                 else:
#                     print("⚠️ Failed to unarchive the page.")
#                 break
#     else:
#         print(f"❌ No page found with title containing '{search_title}'")
# else:
#     print(f"❌ No page found with title containing '{search_title}'")


❌ No page found with title containing 'Updated Task Name'


In [35]:
#@title 🔍 Search by Title → Update Page Status or Tags
database_id = NOTION_DB_ID
title_query = "Updated Task Name"  #@param {type:"string"}
new_status = "Backlog"  #@param {type:"string"}
new_tags = ["Test"]  #@param

database_id = clean_id(database_id)

# Step 1: Search matching pages by title
response = safe_run(notion.databases.query,
    database_id=database_id,
    filter={
        "property": "Name",
        "title": {
            "contains": title_query
        }
    }
)

if response and response.get("results"):
    page = response["results"][0]  # Take the first match
    page_id = page["id"]
    page_title = page["properties"]["Name"]["title"][0]["text"]["content"]

    print(f"✅ Found: {page_title} (ID: {page_id})")

    # Step 2: Update it
    update_fields = {
        "Status": {"select": {"name": new_status}},
        "Tags": {"multi_select": [{"name": tag} for tag in new_tags]}
    }

    update_response = safe_run(notion.pages.update,
        page_id=page_id,
        properties=update_fields
    )

    if update_response:
        print("✅ Page updated:")
        pretty(update_response)

else:
    print("❌ No matching page found.")


✅ Found: Updated Task Name (ID: 239e9c25-ecb0-8119-a7d5-c5b7222b7f00)
✅ Page updated:
{
  "object": "page",
  "id": "239e9c25-ecb0-8119-a7d5-c5b7222b7f00",
  "created_time": "2025-07-23T18:52:00.000Z",
  "last_edited_time": "2025-07-23T19:02:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-08-10",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "select": {
        "id": "ef25b0c9-58fd-4138-8f8c-8cf70d80bc3f",
        "name": "Backlog",
        "color": "gray"
      }


In [ ]:
#@title 📅 Create Daily Journal Page
from datetime import date

database_id =NOTION_DB_ID
tags = ["Journal"]  #@param
status = "Done"  #@param {type:"string"}

database_id = clean_id(database_id)
today_str = date.today().strftime("%Y-%m-%d")

response = safe_run(notion.pages.create,
    parent={"database_id": database_id},
    properties={
        "Name": {"title": [{"text": {"content": f"Journal - {today_str}"}}]},
        "Tags": {"multi_select": [{"name": tag} for tag in tags]},
        "Status": {"select": {"name": status}},
        "Due": {"date": {"start": today_str}}
    }
)

if response:
    print(f"✅ Journal entry created for {today_str}")
    pretty(response)


In [36]:
#@title 📅 Create Daily Journal Page
from datetime import date

database_id = NOTION_DB_ID
status = "Done"  #@param {type:"string"}

database_id = clean_id(database_id)
today_str = date.today().strftime("%Y-%m-%d")

response = safe_run(notion.pages.create,
    parent={"database_id": database_id},
    properties={
        "Name": {
            "title": [
                {"text": {"content": f"Journal - {today_str}"}}
            ]
        },
        "Tags": {
            "multi_select": [{"name": tag} for tag in tags]
        },
        "Status": {
            "select": {"name": status}
        },
        "Due": {
            "date": {"start": today_str}
        }
    }
)

if response:
    print(f"✅ Journal entry created for {today_str}")
    pretty(response)
else:
    print("⚠️ Failed to create journal entry.")


✅ Journal entry created for 2025-07-23
{
  "object": "page",
  "id": "239e9c25-ecb0-8181-8182-e3f0bb84e930",
  "created_time": "2025-07-23T19:02:00.000Z",
  "last_edited_time": "2025-07-23T19:02:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-07-23",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "select": {
        "id": "9456af81-f09a-4f36-91b5-18267bec9048",
        "name": "Done",
        "color": "green"
      }
    },
    "Tags": {
      "id": "ztBX",
      "t

In [37]:
#@title 🏁 Find Task by Title and Mark as Done
database_id =NOTION_DB_ID
task_title = "Journal - 2025-07-23"  #@param {type:"string"}

database_id = clean_id(database_id)

response = safe_run(notion.databases.query,
    database_id=database_id,
    filter={
        "property": "Name",
        "title": {"contains": task_title}
    }
)

if response and response.get("results"):
    page = response["results"][0]
    page_id = page["id"]
    title = page["properties"]["Name"]["title"][0]["text"]["content"]
    print(f"✅ Found: {title} (ID: {page_id})")

    update = safe_run(notion.pages.update,
        page_id=page_id,
        properties={
            "Status": {"select": {"name": "Done"}}
        }
    )

    if update:
        print("✅ Task marked as Done.")
        pretty(update)
else:
    print("❌ Task not found.")


✅ Found: Journal - 2025-07-23 (ID: 239e9c25-ecb0-8181-8182-e3f0bb84e930)
✅ Task marked as Done.
{
  "object": "page",
  "id": "239e9c25-ecb0-8181-8182-e3f0bb84e930",
  "created_time": "2025-07-23T19:02:00.000Z",
  "last_edited_time": "2025-07-23T19:03:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-07-23",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "select": {
        "id": "9456af81-f09a-4f36-91b5-18267bec9048",
        "name": "Done",
        "color": "green"


In [38]:
#@title 🖼️ Update Page Banner (Cover) and Icon
page_id = NOTION_PAGE_ID
cover_url = "https://images.unsplash.com/photo-1503264116251-35a269479413"  #@param {type:"string"}
icon_emoji = "📝"  #@param {type:"string"}

page_id = clean_id(page_id)

response = safe_run(notion.pages.update,
    page_id=page_id,
    cover={
        "type": "external",
        "external": {"url": cover_url}
    },
    icon={
        "type": "emoji",
        "emoji": icon_emoji
    }
)

if response:
    print("✅ Page icon and banner updated.")
    pretty(response)
else:
    print("⚠️ Failed to update cover or icon.")


✅ Page icon and banner updated.
{
  "object": "page",
  "id": "239e9c25-ecb0-8181-8182-e3f0bb84e930",
  "created_time": "2025-07-23T19:02:00.000Z",
  "last_edited_time": "2025-07-23T19:04:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": {
    "type": "external",
    "external": {
      "url": "https://images.unsplash.com/photo-1503264116251-35a269479413"
    }
  },
  "icon": {
    "type": "emoji",
    "emoji": "\ud83d\udcdd"
  },
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-efdd211854d6"
  },
  "archived": false,
  "in_trash": false,
  "properties": {
    "Due": {
      "id": "AnED",
      "type": "date",
      "date": {
        "start": "2025-07-23",
        "end": null,
        "time_zone": null
      }
    },
    "Status": {
      "id": "%60%3FPo",
      "type": "select",
      "sel

In [40]:
#@title 🧰 Utility: Block Creation Functions
def create_rich_text(text):
    return [{"type": "text", "text": {"content": text}}]

def add_bookmark(url: str, caption: str = ""):
    return {
        "object": "block",
        "type": "bookmark",
        "bookmark": {"url": url, "caption": create_rich_text(caption)},
    }

def add_quote(text: str):
    return {
        "object": "block",
        "type": "quote",
        "quote": {"rich_text": create_rich_text(text)},
    }

def add_text(text: str):
    return {
        "object": "block",
        "type": "paragraph",
        "paragraph": {"rich_text": create_rich_text(text)},
    }

def add_heading_1(text: str):
    return {
        "object": "block",
        "type": "heading_1",
        "heading_1": {"rich_text": create_rich_text(text)},
    }

def add_heading_2(text: str):
    return {
        "object": "block",
        "type": "heading_2",
        "heading_2": {"rich_text": create_rich_text(text)},
    }

def add_heading_3(text: str):
    return {
        "object": "block",
        "type": "heading_3",
        "heading_3": {"rich_text": create_rich_text(text)},
    }

def add_numbered_list_item(text: str):
    return {
        "object": "block",
        "type": "numbered_list_item",
        "numbered_list_item": {"rich_text": create_rich_text(text)},
    }

def add_todo_item(text: str, checked: bool = False):
    return {
        "object": "block",
        "type": "to_do",
        "to_do": {"rich_text": create_rich_text(text), "checked": checked},
    }

def add_toggle_list_item(text: str):
    return {
        "object": "block",
        "type": "toggle",
        "toggle": {"rich_text": create_rich_text(text)},
    }

def add_callout(text: str, icon_emoji: str = "💡"):
    return {
        "object": "block",
        "type": "callout",
        "callout": {
            "rich_text": create_rich_text(text),
            "icon": {"type": "emoji", "emoji": icon_emoji},
        },
    }

def add_divider():
    return {"object": "block", "type": "divider", "divider": {}}

def add_table(table_width: int, has_column_header: bool, has_row_header: bool, rows: list):
    return {
        "object": "block",
        "type": "table",
        "table": {
            "table_width": table_width,
            "has_column_header": has_column_header,
            "has_row_header": has_row_header,
            "children": rows,
        },
    }

def add_table_row(cells: list):
    return {
        "type": "table_row",
        "table_row": {
            "cells": [[{"type": "text", "text": {"content": cell}}] for cell in cells]
        },
    }

def add_link_to_page(page_id: str):
    return {
        "object": "block",
        "type": "link_to_page",
        "link_to_page": {"type": "page_id", "page_id": page_id},
    }

def add_video(url: str):
    return {
        "object": "block",
        "type": "video",
        "video": {"type": "external", "external": {"url": url}},
    }

def add_audio(url: str):
    return {
        "object": "block",
        "type": "audio",
        "audio": {"type": "external", "external": {"url": url}},
    }

def add_image(url: str):
    return {
        "object": "block",
        "type": "image",
        "image": {"type": "external", "external": {"url": url}},
    }

def add_code(code: str, language: str):
    return {
        "object": "block",
        "type": "code",
        "code": {
            "rich_text": create_rich_text(code),
            "language": language,
        },
    }

def add_file(url: str, caption: str = ""):
    return {
        "object": "block",
        "type": "file",
        "file": {
            "type": "external",
            "external": {"url": url},
            "caption": create_rich_text(caption),
        },
    }

def add_child_database(title: str):
    return {
        "object": "block",
        "type": "child_database",
        "child_database": {"title": title},
    }

In [46]:
#@title ➕ Add Content Blocks to a Page (Troubleshooting 504 Error)
# Make sure your `notion` client is already initialized from a previous cell
# notion = Client(auth=NOTION_TOKEN)

page_id = NOTION_PAGE_ID
page_id = clean_id(page_id)

# Example of creating a table
table_rows = [
    add_table_row(["Header 1", "Header 2"]),
    add_table_row(["Cell 1", "Cell 2"]),
]

# A list of different block types to add
blocks_to_add = [
    add_heading_1("This is a heading"),
    add_text("This is a paragraph."),
    add_bookmark("https://www.notion.so", "Official Notion Website"),
    add_quote("This is a quote."),
    add_numbered_list_item("First item"),
    add_numbered_list_item("Second item"),
    add_todo_item("To-do item"),
    add_toggle_list_item("Toggle item"),
    add_callout("This is a callout!"),
    add_divider(),
    add_table(2, True, False, table_rows),

    # --- TROUBLESHOOTING STEP ---
    # The video block is the most likely cause of a 504 timeout.
    # It has been commented out to test if the other blocks work.
    # add_video("https://www.youtube.com/watch?v=oHg5SJYRHA0"),

    add_code("print('Hello, World!')", "python"),

    # Using your new, more reliable placeholder link
    add_image("https://placehold.co/400x400.png"),
]

# Append the blocks to the specified page
response = safe_run(notion.blocks.children.append,
    block_id=page_id,
    children=blocks_to_add
)

if response:
    print(f"✅ Successfully added {len(blocks_to_add)} blocks to the page.")
    pretty(response)
else:
    print("⚠️ Failed to add blocks to the page.")

✅ Successfully added 13 blocks to the page.
{
  "object": "list",
  "results": [
    {
      "object": "block",
      "id": "239e9c25-ecb0-81e0-ba6f-d3c2b2b97b38",
      "parent": {
        "type": "page_id",
        "page_id": "239e9c25-ecb0-8181-8182-e3f0bb84e930"
      },
      "created_time": "2025-07-23T19:17:00.000Z",
      "last_edited_time": "2025-07-23T19:17:00.000Z",
      "created_by": {
        "object": "user",
        "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
      },
      "last_edited_by": {
        "object": "user",
        "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
      },
      "has_children": false,
      "archived": false,
      "in_trash": false,
      "type": "heading_1",
      "heading_1": {
        "rich_text": [
          {
            "type": "text",
            "text": {
              "content": "This is a heading",
              "link": null
            },
            "annotations": {
              "bold": false,
              "italic": false,
       

In [50]:
#@title 📁 Create a New Database (with Schema)
# This cell creates a brand new database with a defined set of properties.
# Note: This only creates the database structure and a default table view.
# Additional views (Board, List, etc.) must be added manually in the Notion UI.

parent_page_id = NOTION_PAGE_ID
db_title = "My New Project Tracker"  #@param {type:"string"}

# --- Sanitize Inputs ---
parent_page_id = clean_id(parent_page_id)

# --- Define the Database Schema ---
# This is where you define the columns/properties of your database.
database_schema = {
    # The 'Name' or 'Title' property is required.
    "Name": {
        "title": {}
    },
    "Status": {
        "select": {
            "options": [
                {"name": "Not Started", "color": "gray"},
                {"name": "In Progress", "color": "blue"},
                {"name": "Completed", "color": "green"}
            ]
        }
    },
    "Priority": {
        "multi_select": {
            "options": [
                {"name": "High", "color": "red"},
                {"name": "Medium", "color": "yellow"},
                {"name": "Low", "color": "green"}
            ]
        }
    },
    "Due Date": {
        "date": {}
    },
    "Assignee": {
        "people": {}
    }
}

# --- Create the Database ---
print(f"⚙️ Creating database '{db_title}'...")
response = safe_run(notion.databases.create,
    parent={"type": "page_id", "page_id": parent_page_id},
    title=[{"type": "text", "text": {"content": db_title}}],
    properties=database_schema,
    # is_inline: true makes it a block on the page, false makes it a sub-page
    is_inline=True
)

if response:
    db_url = response.get("url")
    print("\n✅ Database created successfully!")
    print(f"🔗 URL: {db_url}")
    print("\nNext Steps:")
    print("1. Go to the URL above.")
    print("2. Click the '+' next to the 'Table' view to add new views (Board, List, etc.).")
    pretty(response)
else:
    print("\n⚠️ Failed to create the database.")

⚙️ Creating database 'My New Project Tracker'...

✅ Database created successfully!
🔗 URL: https://www.notion.so/239e9c25ecb0816b9fc8d0e01e573a65

Next Steps:
1. Go to the URL above.
2. Click the '+' next to the 'Table' view to add new views (Board, List, etc.).
{
  "object": "database",
  "id": "239e9c25-ecb0-816b-9fc8-d0e01e573a65",
  "cover": null,
  "icon": null,
  "created_time": "2025-07-23T19:24:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_time": "2025-07-23T19:24:00.000Z",
  "title": [
    {
      "type": "text",
      "text": {
        "content": "My New Project Tracker",
        "link": null
      },
      "annotations": {
        "bold": false,
        "italic": false,
        "strikethrough": false,
        "underline": false,
        "code": false,
        "color": "default"
      },
      "plain_text": "

In [79]:
#@title 🧰 Utility: Universal File Upload (Local or URL) - FINAL CORRECTED VERSION
import os
import mimetypes
import requests
from urllib.parse import urlparse

def upload_file_to_page_final(notion_client, parent_page_id: str, source_path_or_url: str):
    """
    Uploads a file from a local path OR a public URL to a Notion page.
    This FINAL version correctly follows the official 2-step Notion API documentation.

    Args:
        notion_client: An initialized Notion client instance.
        parent_page_id: The ID of the page to add the file block to.
        source_path_or_url: The local file path or the public URL of the file.

    Returns:
        The response from the block creation API call, or None if an error occurred.
    """
    filename = ""
    content_type = "application/octet-stream"
    file_data = None

    is_url = source_path_or_url.lower().startswith(('http://', 'https://'))

    # Part 1: Get file data and metadata from either a URL or a local path
    if is_url:
        print("Source is a URL. Fetching file...")
        try:
            file_response = requests.get(source_path_or_url, timeout=30)
            file_response.raise_for_status()
            file_data = file_response.content

            # Determine filename and content type
            content_type = file_response.headers.get('Content-Type', 'application/octet-stream')
            path = urlparse(source_path_or_url).path
            filename = os.path.basename(path)
            if 'octet-stream' in content_type:
                guessed_type, _ = mimetypes.guess_type(filename)
                if guessed_type: content_type = guessed_type

        except requests.exceptions.RequestException as e:
            print(f"⛔ Error fetching URL: {e}")
            return None
    else:
        print("Source is a local file path.")
        if not os.path.exists(source_path_or_url):
            print(f"⛔ Error: File not found at '{source_path_or_url}'")
            return None

        # Determine filename and content type from local file
        filename = os.path.basename(source_path_or_url)
        with open(source_path_or_url, 'rb') as f:
            file_data = f.read()
        mime_type, _ = mimetypes.guess_type(source_path_or_url)
        if mime_type: content_type = mime_type

    print(f"Starting upload for: {filename}")
    notion_token = notion_client.options.auth
    if not notion_token:
        print("⛔ Error: Could not find auth token in notion_client instance.")
        return None

    # Part 2: Execute the 3-step Notion API upload process
    try:
        # Step 1: Prepare the upload with Notion. This gets us a temporary URL.
        prepare_url = "https://api.notion.com/v1/file_uploads"
        print(f"  [1/3] Preparing upload with Notion...")
        prepare_headers = {"Authorization": f"Bearer {notion_token}", "Notion-Version": "2022-06-28", "Content-Type": "application/json"}
        prepare_data = {"filename": filename, "content_type": content_type}

        prepare_response = requests.post(prepare_url, headers=prepare_headers, json=prepare_data)
        prepare_response.raise_for_status()
        upload_details = prepare_response.json()
        upload_url = upload_details["upload_url"]
        file_upload_id = upload_details["id"]
        print("    ✅ Got temporary upload URL.")

        # Step 2: Upload the actual file data to the temporary URL using multipart/form-data.
        print("  [2/3] Uploading file data...")
        files_payload = {'file': (filename, file_data, content_type)}
        # We do NOT set the Content-Type header here, as `requests` does it for us.
        upload_response = requests.post(upload_url, files=files_payload)
        upload_response.raise_for_status()
        print("    ✅ File data sent successfully.")

        # Step 3: Attach the uploaded file to a new block on the target page.
        print("  [3/3] Attaching file to page...")
        file_block_data = {
            "type": "file",
            "file": {
                "type": "file_upload",
                "file_upload": {"id": file_upload_id},
                "caption": [{"type": "text", "text": {"content": filename}}]
            }
        }
        attach_response = safe_run(notion_client.blocks.children.append, block_id=parent_page_id, children=[file_block_data])
        if attach_response:
            print("    ✅ File block created on page.")
            return attach_response
        return None

    except requests.exceptions.HTTPError as e:
        print(f"⛔ API HTTP Error: {e}")
        if e.response:
            print(f"    Status Code: {e.response.status_code}")
            print(f"    Response Body: {e.response.text}")
        return None
    except Exception as e:
        print(f"⛔ An unexpected error occurred: {e}")
        return None

print("✅ Universal file upload utility function `upload_file_to_page_final` is now defined.")

✅ Universal file upload utility function `upload_file_to_page_final` is now defined.


In [96]:
# --- 1. UPDATED File Upload Utility ---
def upload_file_and_get_id(notion_client, source_path_or_url: str):
    """
    Uploads a file but DOES NOT attach it to a page.
    Instead, it returns the file_upload_id needed for other operations like comments.

    Returns:
        The file_upload_id (str) on success, or None on failure.
    """
    filename, content_type, file_data = "", "application/octet-stream", None
    is_url = source_path_or_url.lower().startswith(('http://', 'https://'))

    if is_url:
        print(f"Source is a URL. Fetching file...")
        # (Error handling and data fetching logic is the same as before)
        try:
            file_response = requests.get(source_path_or_url, timeout=30)
            file_response.raise_for_status()
            file_data = file_response.content
            content_type = file_response.headers.get('Content-Type', 'application/octet-stream')
            path = urlparse(source_path_or_url).path
            filename = os.path.basename(path)
            if 'octet-stream' in content_type:
                guessed_type, _ = mimetypes.guess_type(filename)
                if guessed_type: content_type = guessed_type
        except requests.exceptions.RequestException as e:
            print(f"⛔ Error fetching URL: {e}")
            return None
    else: # Local file
        # (Error handling and data fetching logic is the same as before)
        print("Source is a local file path.")
        if not os.path.exists(source_path_or_url):
            print(f"⛔ Error: File not found at '{source_path_or_url}'")
            return None
        filename = os.path.basename(source_path_or_url)
        with open(source_path_or_url, 'rb') as f: file_data = f.read()
        mime_type, _ = mimetypes.guess_type(source_path_or_url)
        if mime_type: content_type = mime_type

    print(f"Starting upload for: {filename}")
    notion_token = notion_client.options.auth
    if not notion_token:
        print("⛔ Error: Could not find auth token in notion_client instance.")
        return None

    try:
        prepare_url = "https://api.notion.com/v1/file_uploads"
        print("  [1/2] Preparing upload with Notion...")
        prepare_headers = {"Authorization": f"Bearer {notion_token}", "Notion-Version": "2022-06-28", "Content-Type": "application/json"}
        prepare_data = {"filename": filename, "content_type": content_type}
        prepare_response = requests.post(prepare_url, headers=prepare_headers, json=prepare_data)
        prepare_response.raise_for_status()
        upload_details = prepare_response.json()
        upload_url, file_upload_id = upload_details["upload_url"], upload_details["id"]
        print("    ✅ Got temporary upload URL.")

        print("  [2/2] Uploading file data...")
        files_payload = {'file': (filename, file_data, content_type)}
        upload_response = requests.post(upload_url, files=files_payload)
        upload_response.raise_for_status()
        print("    ✅ File data sent successfully.")

        # Return the ID for use in other functions
        return file_upload_id

    except requests.exceptions.HTTPError as e:
        print(f"⛔ API HTTP Error: {e}")
        if e.response: print(f"    Response Body: {e.response.text}")
        return None
    except Exception as e:
        print(f"⛔ An unexpected error occurred: {e}")
        return None

In [100]:
#@title ⬆️ Execute a File Upload and Attach to Page

# --- Configuration ---
# Use the globally set page ID as the parent for the new file block.
parent_page_id = NOTION_PAGE_ID

# Provide the source of the file to upload.
source = "https://s2.q4cdn.com/175719177/files/doc_presentations/Placeholder-PDF.pdf" #@param {type:"string"}


# --- Check if the parent page ID is set ---
if not parent_page_id:
    print("❌ Cannot attach file. `NOTION_PAGE_ID` is not set.")
    print("   Please run the 'Set Global Page and Database IDs' cell first.")
else:
    # --- Step 1: Upload the file to get its ID ---
    print("--- Step 1: Uploading file to Notion's servers ---")
    # Use the correct, updated utility function
    uploaded_file_id = upload_file_and_get_id(
        notion_client=notion,
        source_path_or_url=source
    )

    # --- Step 2: If upload was successful, attach the file to the page ---
    if uploaded_file_id:
        print("\n--- Step 2: Attaching uploaded file to the page ---")

        # Get the filename from the URL to use as a caption
        filename = os.path.basename(urlparse(source).path)

        # Prepare the file block payload using the ID we received
        file_block_data = {
            "type": "file",
            "file": {
                "type": "file_upload",
                "file_upload": {"id": uploaded_file_id},
                "caption": [{"type": "text", "text": {"content": filename}}]
            }
        }

        # Make the API call to append the block
        response = safe_run(notion.blocks.children.append,
            block_id=parent_page_id,
            children=[file_block_data]
        )

        if response:
            print("\n🎉 Success! File uploaded and attached to Notion page.")
            pretty(response)
        else:
            print("\n⚠️ Operation failed. Could not attach the file to the page.")
    else:
        print("\n⚠️ Operation failed. The initial file upload did not succeed.")

--- Step 1: Uploading file to Notion's servers ---
Source is a URL. Fetching file...
Starting upload for: Placeholder-PDF.pdf
  [1/2] Preparing upload with Notion...
    ✅ Got temporary upload URL.
  [2/2] Uploading file data...
⛔ API HTTP Error: 401 Client Error: Unauthorized for url: https://api.notion.com/v1/file_uploads/239e9c25-ecb0-8137-8fa7-00b298821b04/send

⚠️ Operation failed. The initial file upload did not succeed.


In [81]:
# --- 2. NEW Comment Utility ---
def add_comment_to_page(notion_client, page_id: str, comment_text: str, attachment_ids: list = None, display_name: str = None):
    """
    Creates a new comment on a page with optional attachments and a custom display name.

    Args:
        notion_client: An initialized Notion client instance.
        page_id: The ID of the page to add the comment to.
        comment_text: The text content of the comment.
        attachment_ids: A list of file_upload_id strings from successfully uploaded files.
        display_name: A custom string to show as the commenter's name.
    """
    print(f"💬 Creating comment on page {page_id}...")

    # Build the payload dictionary
    payload = {
        "parent": {"page_id": page_id},
        "rich_text": [{"text": {"content": comment_text}}]
    }

    # Add attachments if provided
    if attachment_ids and isinstance(attachment_ids, list):
        payload["attachments"] = [{"file_upload_id": an_id} for an_id in attachment_ids]
        print(f"  📎 Attaching {len(attachment_ids)} file(s).")

    # Add custom display name if provided
    if display_name:
        payload["display_name"] = {"type": "custom", "custom": {"name": display_name}}
        print(f"  👤 Using custom display name: '{display_name}'")

    # Make the API call
    response = safe_run(notion_client.comments.create, **payload)

    if response:
        print("  ✅ Comment created successfully!")
        return response
    else:
        print("  ⚠️ Failed to create comment.")
        return None

print("✅ Comment and updated file upload utilities are now defined.")

✅ Comment and updated file upload utilities are now defined.


In [84]:
#@title 💬 Execute Comment Creation

page_id_to_comment_on = NOTION_PAGE_ID
page_id_to_comment_on = clean_id(page_id_to_comment_on)

# --- Example 1: Create a simple text-only comment ---
print("--- Running Example 1: Simple Comment ---")
add_comment_to_page(
    notion_client=notion,
    page_id=page_id_to_comment_on,
    comment_text="This is a simple test comment from the API."
)
print("-" * 40)


# --- Example 2: Create a comment with a custom name ---
print("\n--- Running Example 2: Comment with Custom Name ---")
add_comment_to_page(
    notion_client=notion,
    page_id=page_id_to_comment_on,
    comment_text="This comment should appear from a custom bot name.",
    display_name="Colab Bot"
)
print("-" * 40)


# --- Example 3: Create a comment with a file attachment ---
print("\n--- Running Example 3: Comment with Attachment ---")
# First, upload a file to get its ID
file_source_url = "https://s2.q4cdn.com/175719177/files/doc_presentations/Placeholder-PDF.pdf"
uploaded_file_id = upload_file_and_get_id(notion, file_source_url)

# IMPORTANT: Only proceed if the file upload was successful
if uploaded_file_id:
    # Now, create the comment and pass the ID in a list
    comment_response = add_comment_to_page(
        notion_client=notion,
        page_id=page_id_to_comment_on,
        comment_text="Please review the attached file.",
        attachment_ids=[uploaded_file_id] # Pass the ID in a list
    )
    if comment_response:
        pretty(comment_response)
else:
    print("⚠️ Skipping comment with attachment because file upload failed.")
print("-" * 40)

--- Running Example 1: Simple Comment ---
💬 Creating comment on page 239e9c25ecb081818182e3f0bb84e930...
⛔ Error: Insufficient permissions for this endpoint.
  ⚠️ Failed to create comment.
----------------------------------------

--- Running Example 2: Comment with Custom Name ---
💬 Creating comment on page 239e9c25ecb081818182e3f0bb84e930...
  👤 Using custom display name: 'Colab Bot'
⛔ Error: Insufficient permissions for this endpoint.
  ⚠️ Failed to create comment.
----------------------------------------

--- Running Example 3: Comment with Attachment ---
Source is a URL. Fetching file...
Starting upload for: Placeholder-PDF.pdf
  [1/2] Preparing upload with Notion...
    ✅ Got temporary upload URL.
  [2/2] Uploading file data...
⛔ API HTTP Error: 401 Client Error: Unauthorized for url: https://api.notion.com/v1/file_uploads/239e9c25-ecb0-81da-891a-00b22d1ded41/send
⚠️ Skipping comment with attachment because file upload failed.
----------------------------------------


In [98]:
#@title 🧰 Utilities: Users (Corrected Bot Detection)

def list_all_users(notion_client):
    """
    Lists all users (people and bots) that the integration can access.
    """
    print("Listing all users...")
    response = safe_run(notion_client.users.list)

    if response and response.get("results"):
        print(f"✅ Found {len(response['results'])} user(s):")
        for user in response["results"]:
            user_type = user.get("type", "unknown")
            user_name = user.get("name", "Unknown Name")
            user_id = user.get("id")
            print(f"  - {user_name} (Type: {user_type}, ID: {user_id})")
        # The pretty print shows the full detail
        # pretty(response)
        return response
    else:
        print("⚠️ Could not list users or no users found.")
        return None


def get_user_by_id(notion_client, user_id: str):
    """
    Retrieves the details for a single user by their unique ID.
    """
    print(f"🔎 Retrieving user with ID: {user_id}...")
    response = safe_run(notion_client.users.retrieve, user_id=user_id)

    if response:
        user_name = response.get("name", "Unknown Name")
        print(f"✅ Found user: {user_name}")
        pretty(response)
        return response
    else:
        print(f"⚠️ Could not retrieve user with ID: {user_id}")
        return None


def get_bot_details(notion_client):
    """
    Retrieves the details for the integration's own bot user by finding
    the unique bot in the user list with owner information.
    """
    print("🤖 Retrieving details for this integration (bot)...")

    all_users_response = safe_run(notion_client.users.list)

    if all_users_response and all_users_response.get("results"):
        # The current integration is the only bot that will have a populated 'owner' field.
        # Other bots will have an empty 'bot: {}' object. We search for the one that has it.
        current_bot_user = next(
            (user for user in all_users_response["results"]
             if user.get("type") == "bot" and user.get("bot", {}).get("owner")),
            None
        )

        if current_bot_user:
            print("✅ Found Bot Details:")
            pretty(current_bot_user)
            return current_bot_user

    print("⚠️ Could not retrieve bot details automatically. This can happen if the integration has very limited permissions.")
    return None


print("✅ User utility functions (with corrected bot detection) are now defined.")

✅ User utility functions (with corrected bot detection) are now defined.


In [99]:
#@title 🧑‍💻 Execute User Utilities (Self-Contained)

# --- Example 1: List all available users and store the result ---
print("--- Running Example 1: List All Users ---")
# We capture the response so we can use it in the next step
all_users_response = list_all_users(notion)
print("-" * 40)


# --- Example 2: Retrieve a specific user by their ID ---
# This example now automatically finds a valid ID from the list above.
print("\n--- Running Example 2: Get User by ID ---")
user_id_to_find = ""

# Check if the previous API call was successful and returned users
if all_users_response and all_users_response.get("results"):

    # Try to find the first user that is a 'person'
    person_user = next((user for user in all_users_response["results"] if user.get("type") == "person"), None)

    if person_user:
        # If we found a person, use their ID
        user_id_to_find = person_user.get("id")
        print(f"✅ Automatically selected 'person' user to find: {person_user.get('name')}")
    else:
        # If no person was found, just use the first user in the list (likely a bot)
        first_user = all_users_response["results"][0]
        user_id_to_find = first_user.get("id")
        print(f"✅ No 'person' found. Automatically selected first user: {first_user.get('name')}")

# Now, run the get_user_by_id function with the ID we found
if user_id_to_find:
    get_user_by_id(notion, user_id=user_id_to_find)
else:
    print("⚠️ Skipping example. Could not find any users from the list.")
print("-" * 40)


# --- Example 3: Get the details of the integration (bot) itself ---
print("\n--- Running Example 3: Get Bot Details ---")
get_bot_details(notion)
print("-" * 40)

--- Running Example 1: List All Users ---
Listing all users...
✅ Found 3 user(s):
  - Tyrique Daniel (Type: person, ID: 3326814f-5914-4d6e-bc68-e619eaaf657e)
  - CyberOni (Type: bot, ID: f3e8fc01-40b0-4c02-a33f-cd0b71411599)
  - DealScale (Type: bot, ID: 4e99c9e5-fb26-4f6f-84c5-5866edb12a95)
----------------------------------------

--- Running Example 2: Get User by ID ---
✅ Automatically selected 'person' user to find: Tyrique Daniel
🔎 Retrieving user with ID: 3326814f-5914-4d6e-bc68-e619eaaf657e...
✅ Found user: Tyrique Daniel
{
  "object": "user",
  "id": "3326814f-5914-4d6e-bc68-e619eaaf657e",
  "name": "Tyrique Daniel",
  "avatar_url": null,
  "type": "person",
  "person": {
    "email": "codingoni@gmail.com"
  },
  "request_id": "dbc60c3b-8808-43f5-bb3a-bbe21d5221b2"
}
----------------------------------------

--- Running Example 3: Get Bot Details ---
🤖 Retrieving details for this integration (bot)...
✅ Found Bot Details:
{
  "object": "user",
  "id": "4e99c9e5-fb26-4f6f-84c5-5

In [103]:
#@title 🧰 Utilities: Webhook Processor and Page Retrieval (Corrected)

def get_page_by_id(notion_client, page_id: str):
    """
    Retrieves the details for a single page by its unique ID.
    """
    print(f"🔎 Retrieving page with ID: {page_id}...")
    response = safe_run(notion_client.pages.retrieve, page_id=page_id)

    if response:
        # Try to get the page title from the properties
        properties = response.get("properties", {})
        # Find the property that has the type 'title'
        title_prop_name = next((prop for prop, details in properties.items() if details.get("type") == "title"), None)

        if title_prop_name:
            title_text = properties[title_prop_name]["title"][0]["plain_text"]
            print(f"✅ Found page: '{title_text}'")
        else:
            print("✅ Found page, but it has no title property.")

        pretty(response)
        return response
    else:
        print(f"⚠️ Could not retrieve page with ID: {page_id}")
        return None


def process_webhook_payload(notion_client, payload: dict):
    """
    Parses a simulated webhook payload and takes the appropriate follow-up action.
    """
    try:
        event_type = payload.get("type", "unknown.event")
        entity = payload.get("entity", {})
        entity_id = entity.get("id")
        entity_type = entity.get("type")
        timestamp = payload.get("timestamp")

        print(f"🔔 Received webhook event: '{event_type}'")
        print(f"   - For: {entity_type} (ID: {entity_id})")
        print(f"   - At: {timestamp}")

        if not entity_id:
            print("   ⚠️ No entity ID found in payload. Cannot proceed.")
            return

        # --- Take action based on the entity type ---
        if entity_type == "page":
            print(f"\n   ➡️ Action: Fetching latest details for the page...")
            # --- THIS IS THE CORRECTED LINE ---
            page_details = get_page_by_id(notion_client, entity_id)
            if not page_details:
                print(f"      Could not retrieve page {entity_id}. It may have been deleted.")

        elif entity_type == "database":
            print(f"\n   ➡️ Action: Fetching latest details for the database...")
            db_details = safe_run(notion_client.databases.retrieve, database_id=entity_id)
            if db_details:
                pretty(db_details)
            else:
                print(f"      Could not retrieve database {entity_id}. It may have been deleted.")

        elif entity_type == "comment":
            parent_id = payload.get("data", {}).get("parent", {}).get("id")
            if parent_id:
                print(f"\n   ➡️ Action: Fetching comment thread from parent (ID: {parent_id})...")
                comment_thread = safe_run(notion_client.comments.list, block_id=parent_id)
                if comment_thread:
                    print(f"      Found {len(comment_thread.get('results', []))} comment(s) in the thread.")
                    pretty(comment_thread)
            else:
                print("   ⚠️ Comment event received, but no parent ID was found in the data.")

        else:
            print(f"\n   ➡️ No specific action defined for entity type: '{entity_type}'")

    except Exception as e:
        print(f"⛔ An error occurred while processing the payload: {e}")


print("✅ Corrected webhook and page retrieval utilities are now defined.")

✅ Corrected webhook and page retrieval utilities are now defined.


In [104]:
#@title 📡 Execute Webhook Simulation

# --- Sample Payloads from Notion Documentation ---
# I have replaced the placeholder IDs with your global variables
# so the follow-up API calls will work.

sample_payloads = {
    "page.properties_updated": {
      "id": "1782edd6-a853-4d4a-b02c-9c8c16f28e53",
      "timestamp": "2024-12-05T23:57:05.379Z",
      "workspace_id": "13950b26-c203-4f3b-b97d-93ec06319565",
      "type": "page.properties_updated",
      "entity": { "id": NOTION_PAGE_ID, "type": "page" } # Using your global page ID
    },
    "database.schema_updated": {
      "id": "5496f509-6988-4bab-b6a9-bdce0b720ca0",
      "timestamp": "2024-12-05T23:55:22.243Z",
      "workspace_id": "13950b26-c203-4f3b-b97d-93ec06319565",
      "type": "database.schema_updated",
      "entity": { "id": NOTION_DB_ID, "type": "database" } # Using your global DB ID
    },
    "comment.created": {
      "id": "c6780f24-10b7-4f42-a6fd-230b6cf7ad69",
      "timestamp": "2024-12-05T20:46:45.854Z",
      "workspace_id": "13950b26-c203-4f3b-b97d-93ec06319565",
      "type": "comment.created",
      "entity": { "id": "some-comment-id", "type": "comment" },
      "data": { "parent": { "id": NOTION_PAGE_ID, "type": "page" } } # Parent is your global page
    }
}

# --- Simulation Configuration ---
event_to_simulate = "page.properties_updated"  #@param ["page.properties_updated", "database.schema_updated", "comment.created"]

# --- Run the Simulation ---
print(f"--- Simulating a '{event_to_simulate}' event ---")
selected_payload = sample_payloads.get(event_to_simulate)

if selected_payload:
    # Check if the necessary ID is set
    entity_id_to_check = selected_payload.get("entity", {}).get("id")
    if not entity_id_to_check:
         print(f"⚠️ Cannot run simulation. The global variable for the entity ID (e.g., NOTION_PAGE_ID) is not set.")
    else:
        process_webhook_payload(notion, selected_payload)
else:
    print("❌ Invalid event type selected.")

print("-" * 40)

--- Simulating a 'page.properties_updated' event ---
🔔 Received webhook event: 'page.properties_updated'
   - For: page (ID: 239e9c25ecb081818182e3f0bb84e930)
   - At: 2024-12-05T23:57:05.379Z

   ➡️ Action: Fetching latest details for the page...
🔎 Retrieving page with ID: 239e9c25ecb081818182e3f0bb84e930...
✅ Found page: 'Journal - 2025-07-23'
{
  "object": "page",
  "id": "239e9c25-ecb0-8181-8182-e3f0bb84e930",
  "created_time": "2025-07-23T19:02:00.000Z",
  "last_edited_time": "2025-07-23T19:51:00.000Z",
  "created_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "last_edited_by": {
    "object": "user",
    "id": "4e99c9e5-fb26-4f6f-84c5-5866edb12a95"
  },
  "cover": {
    "type": "external",
    "external": {
      "url": "https://images.unsplash.com/photo-1503264116251-35a269479413"
    }
  },
  "icon": {
    "type": "emoji",
    "emoji": "\ud83d\udcdd"
  },
  "parent": {
    "type": "database_id",
    "database_id": "239e9c25-ecb0-81ee-ae1b-e

# ⚠️ Notion API Limitations & Notes

### General
- You **cannot** modify a database's property schema (e.g., add or remove properties) via the API after it's been created.
- Pages can only be **archived (soft deleted)**, not permanently deleted.
- Rate limits apply; avoid sending many requests in a short time.
- Creating and updating select/multi-select properties requires the options to already exist in the database schema. You cannot create new options on the fly.
- Querying large databases or block lists requires handling pagination using the `start_cursor` parameter.

### Databases & Views
- The API **cannot** create specific view types like Board, Gallery, Calendar, or Timeline. When a database is created via the API, it **always** defaults to a single Table view.
- You **cannot** create a "Linked Database" block via the API. This must be done manually in the Notion UI.
- The workaround is to create a database with the necessary *properties* (e.g., a 'Date' property for a Calendar, a 'Select' property for a Board) and then add the desired views manually in the Notion app.

### File & Image Uploads
- When adding an `image` block, the URL must be a direct, publicly accessible link to the image file (e.g., ending in `.png`, `.jpg`). Incorrect URLs will be rejected.
- External media blocks (images, videos) can fail with a `504 Gateway Timeout` error if the source server is slow to respond.
- Securely uploading a file (from a local path or a URL) is a **multi-step process**:
    1.  **Prepare** the upload with Notion to get a temporary URL.
    2.  **Upload** the file data to that temporary URL.
    3.  **Attach** the file by referencing its `file_upload_id` in a new block.

### Comments
- To create comments, the integration **must** have the **'Insert comments'** capability enabled in its settings page on the Notion website. A `401 Unauthorized` or `Insufficient permissions` error means this permission is missing.
- To attach a file to a comment, you must first complete the file upload process to get a `file_upload_id` to use in the comment payload.